# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to serzzh@gmail.com and will expire on March 13, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489309716.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.897264     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.963699     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
s = model_all.get('coefficients')

In [7]:
s.show()

Canvas is accessible via web browser at the URL: http://localhost:44657/index.html
Opening Canvas in default web browser.


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np 

In [10]:
j=np.logspace(1, 7, num=13)

In [11]:
j[12]

10000000.0

In [12]:
RSS=range(len(j)+1)

In [13]:
model0 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[0])
model1 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[1])
model2 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[2])
model3 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[3])
model4 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[4])
model5 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[5])
model6 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[6])
model7 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[7])
model8 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[8])
model9 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[9])
model10 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[10])
model11 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[11])
model12 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=j[12])

    

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.677708     | 5740519.699879     | 289427.896727 |

| 6         | 7        | 0.000003  | 0.724351     | 5659149.988446     | 285408.885485 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [14]:
model13 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=2e9)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.757752     | 6273049.924496     | 322574.149756 |

| 6         | 7        | 0.000003  | 0.814058     | 6241845.481483     | 321002.445033 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [15]:
print model12.get('coefficients')
print model13.get('coefficients')

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None |   19604.7163508    |  None  |
|     bedrooms     |  None |   7922.62023075    |  None  |
| bedrooms_square  |  None |   928.112147889    |  None  |
|    bathrooms     |  None |    25392.743443    |  None  |
|   sqft_living    |  None |   39.1083048767    |  None  |
| sqft_living_sqrt |  None |   1123.95233925    |  None  |
|     sqft_lot     |  None | -0.000823987151992 |  None  |
|  sqft_lot_sqrt   |  None |   147.859054391    |  None  |
|      floors      |  None |    21154.353574    |  None  |
|  floors_square   |  None |    12871.042877    |  None  |
+------------------+-------+--------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
+------------------+----

In [16]:
model12.predict(validation)

dtype: float
Rows: 9635
[339584.002686081, 707050.1267636236, 283089.26119813113, 422297.4490814742, 484568.2533733386, 343908.5005864564, 457026.0364054972, 745879.4676320987, 500246.8759116139, 356695.2669341936, 1023281.9239416851, 401522.9753585166, 592803.2985750148, 551738.0292310226, 480599.4091990277, 366862.46277010534, 687910.133064519, 448910.77856123575, 564998.5662158285, 597420.7880447165, 816493.8317898898, 433040.66821627424, 372392.5180538668, 630012.7142461552, 621073.2788473135, 553999.8173848942, 561991.3980927777, 1075004.9705026676, 539668.1422743737, 568698.3509059405, 539479.1580330073, 651867.5776343513, 553196.4878006927, 379560.7076915436, 508302.56404136645, 826456.5745348015, 555788.3657562194, 522130.1694032999, 488611.1136224284, 409894.47593247634, 683705.3555072783, 406008.8608821938, 345656.01403337123, 778275.5337493864, 441534.384622719, 515424.1343265099, 722455.1817137547, 361315.08209299896, 542907.0268523982, 570211.0134376264, 414530.49275963346

In [17]:
validation['price']

dtype: float
Rows: 9635
[221900.0, 538000.0, 180000.0, 510000.0, 257500.0, 291850.0, 229500.0, 662500.0, 395000.0, 230000.0, 2000000.0, 667000.0, 719000.0, 580500.0, 280000.0, 535000.0, 696000.0, 550000.0, 640000.0, 625000.0, 775000.0, 685000.0, 450000.0, 920000.0, 885000.0, 292500.0, 301000.0, 951000.0, 430000.0, 505000.0, 549000.0, 317625.0, 287000.0, 325000.0, 349000.0, 832500.0, 390000.0, 360000.0, 355000.0, 356000.0, 461000.0, 335000.0, 153000.0, 905000.0, 247500.0, 555000.0, 1088000.0, 188500.0, 470000.0, 597750.0, 570000.0, 329950.0, 480000.0, 740500.0, 535000.0, 660000.0, 395000.0, 770000.0, 445000.0, 822500.0, 660500.0, 784000.0, 199000.0, 382500.0, 519950.0, 527700.0, 921500.0, 890000.0, 430000.0, 560000.0, 2250000.0, 350000.0, 215000.0, 650000.0, 320000.0, 247000.0, 438000.0, 900000.0, 441000.0, 807100.0, 371500.0, 272000.0, 313000.0, 917500.0, 673000.0, 385000.0, 269950.0, 330000.0, 260000.0, 589000.0, 1095000.0, 269000.0, 615000.0, 305000.0, 166950.0, 799000.0, 256883.0, 3

In [18]:
    RSS[0] = sum((model0.predict(validation)-validation['price'])**2)
    RSS[1] = sum((model1.predict(validation)-validation['price'])**2)
    RSS[2] = sum((model2.predict(validation)-validation['price'])**2)
    RSS[3] = sum((model3.predict(validation)-validation['price'])**2)
    RSS[4] = sum((model4.predict(validation)-validation['price'])**2)
    RSS[5] = sum((model5.predict(validation)-validation['price'])**2)
    RSS[6] = sum((model6.predict(validation)-validation['price'])**2)
    RSS[7] = sum((model7.predict(validation)-validation['price'])**2)
    RSS[8] = sum((model8.predict(validation)-validation['price'])**2)
    RSS[9] = sum((model9.predict(validation)-validation['price'])**2)
    RSS[10] = sum((model10.predict(validation)-validation['price'])**2)
    RSS[11] = sum((model11.predict(validation)-validation['price'])**2)
    RSS[12] = sum((model12.predict(validation)-validation['price'])**2)
    RSS[13] = sum((model13.predict(validation)-validation['price'])**2)
   

In [19]:
for i in range(len(RSS)):
    print RSS[i]

6.25766285142e+14
6.25766285362e+14
6.25766286058e+14
6.25766288257e+14
6.25766295212e+14
6.25766317206e+14
6.25766386761e+14
6.25766606749e+14
6.25767302792e+14
6.25769507644e+14
6.25776517727e+14
6.25799062845e+14
6.25883719085e+14
8.18340242319e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [20]:
l1_penalty = 10

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [21]:
 model = graphlab.linear_regression.create(testing, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10)
RSS = sum((model.predict(testing)-testing['price'])**2)

Linear regression:

--------------------------------------------------------

Number of examples          : 2217

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000018  | 0.404793     | 2490856.564067     | 277537.432880 |

| 6         | 7        | 0.000018  | 0.430768     | 2480795.822625     | 274447.091383 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [22]:
print RSS

1.55205417245e+14


In [23]:
coef = model.get('coefficients')

In [24]:
coef[5:13]

name,index,value,stderr
sqft_living_sqrt,None,1115.10438309,None
sqft_lot,None,-0.15538020793,None
sqft_lot_sqrt,None,126.735040031,None
floors,None,19806.9697903,None
floors_square,None,12646.6371612,None
waterfront,None,413704.286579,None
view,None,86807.2319587,None
condition,None,5914.49546888,None


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [25]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [26]:
l1_penalty_values = np.logspace(8, 10, num=20)

In [27]:
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [28]:
for l1_penalty in np.logspace(8, 10, num=20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
    print "----------------------------------------------------------"
    print model['coefficients']['value'].nnz()
    print l1_penalty
    print sum((model.predict(validation)-validation['price'])**2)
    print '----------------------------------------------------------'
    


Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

----------------------------------------------------------


This model may not be optimal. To improve it, consider increasing `max_iterations`.

1
10000000000.0
1.22915716064e+15
----------------------------------------------------------


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [29]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [30]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [31]:
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
    print "----------------------------------------------------------"
    print model['coefficients']['value'].nnz()
    print l1_penalty
    print sum((model.predict(validation)-validation['price'])**2)
    print '----------------------------------------------------------'
    

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

+-----------+----------+-----------+--------------+--------------------+---------------+

----------------------------------------------------------


TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

6
3792690190.73
1.08186759232e+15
----------------------------------------------------------


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [32]:
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448968612.16)
    print "----------------------------------------------------------"
    print model['coefficients']['value'].nnz()
    print l1_penalty
    print sum((model.predict(validation)-validation['price'])**2)
    print '----------------------------------------------------------'
 

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

+-----------+----------+-----------+--------------+--------------------+---------------+

----------------------------------------------------------


TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

7
3792690190.73
1.04693748875e+15
----------------------------------------------------------


In [33]:
model.get('coefficients')

name,index,value,stderr
(intercept),None,222253.192544,None
bedrooms,None,661.722717782,None
bedrooms_square,None,0.0,None
bathrooms,None,15873.9572593,None
sqft_living,None,32.4102214513,None
sqft_living_sqrt,None,690.114773313,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None
